## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gmap_key

# Configure gmaps API key
gmaps.configure(api_key=gmap_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
cityData_df = pd.read_csv("../weather_database/WeatherPy_Database.csv")
cityData_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Okhotsk,RU,59.38,143.30,51.62,73,100,8.01,overcast clouds
1,1,Shawnee,US,39.05,-95.77,86.00,39,1,4.70,clear sky
2,2,Avarua,CK,-21.21,-159.78,77.00,83,15,24.16,few clouds
3,3,Aykhal,RU,66.00,111.50,46.40,65,58,4.47,broken clouds
4,4,Hasaki,JP,35.73,140.83,75.00,100,20,2.24,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferredCities_df = cityData_df.loc[(cityData_df["Max Temp"] >= min_temp) & (cityData_df["Max Temp"] <= max_temp)]
preferredCities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Shawnee,US,39.05,-95.77,86.00,39,1,4.70,clear sky
2,2,Avarua,CK,-21.21,-159.78,77.00,83,15,24.16,few clouds
4,4,Hasaki,JP,35.73,140.83,75.00,100,20,2.24,few clouds
6,6,Grand Baie,MU,-20.02,57.58,72.00,62,0,3.00,clear sky
7,7,Ko Samui,TH,9.54,99.94,83.17,72,94,7.81,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferredCities_df.count()

City_ID                397
City                   397
Country                396
Lat                    397
Lng                    397
Max Temp               397
Humidity               397
Cloudiness             397
Wind Speed             397
Current Description    397
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferredCities_df = preferredCities_df.dropna()
preferredCities_df.count()

City_ID                396
City                   396
Country                396
Lat                    396
Lng                    396
Max Temp               396
Humidity               396
Cloudiness             396
Wind Speed             396
Current Description    396
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferredCities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Shawnee,US,86.00,clear sky,39.05,-95.77,
2,Avarua,CK,77.00,few clouds,-21.21,-159.78,
4,Hasaki,JP,75.00,few clouds,35.73,140.83,
6,Grand Baie,MU,72.00,clear sky,-20.02,57.58,
7,Ko Samui,TH,83.17,overcast clouds,9.54,99.94,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000,
          "type": "lodging",
          "key": gmap_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Shawnee,US,86.00,clear sky,39.05,-95.77,Fairfield Inn by Marriott Topeka
2,Avarua,CK,77.00,few clouds,-21.21,-159.78,Paradise Inn
4,Hasaki,JP,75.00,few clouds,35.73,140.83,Inubosaki Kanko Hotel
6,Grand Baie,MU,72.00,clear sky,-20.02,57.58,Canonnier Beachcomber Golf Resort & Spa
7,Ko Samui,TH,83.17,overcast clouds,9.54,99.94,"The Passage Samui Villas & Resort, Thailand"


In [11]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

# replace empty strings with NaNs
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)

# drop empty rows
clean_hotel_df = hotel_df.dropna()

# check total row count
print(clean_hotel_df.count())

clean_hotel_df.head(10)

City                   365
Country                365
Max Temp               365
Current Description    365
Lat                    365
Lng                    365
Hotel Name             365
dtype: int64


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Shawnee,US,86.00,clear sky,39.05,-95.77,Fairfield Inn by Marriott Topeka
2,Avarua,CK,77.00,few clouds,-21.21,-159.78,Paradise Inn
4,Hasaki,JP,75.00,few clouds,35.73,140.83,Inubosaki Kanko Hotel
6,Grand Baie,MU,72.00,clear sky,-20.02,57.58,Canonnier Beachcomber Golf Resort & Spa
7,Ko Samui,TH,83.17,overcast clouds,9.54,99.94,"The Passage Samui Villas & Resort, Thailand"
8,Souillac,MU,69.80,broken clouds,-20.52,57.52,Bils' Villa
9,Colon,PA,81.00,moderate rain,9.36,-79.90,Sand Diamond Hotel
10,Vaini,TO,73.40,light intensity drizzle,-21.20,-175.20,Keleti Beach Resort
11,Jamestown,US,77.00,clear sky,42.10,-79.24,Hampton Inn & Suites Jamestown
12,Les Cayes,HT,85.53,overcast clouds,18.20,-73.75,Villa Orphee


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.3, 31.0), zoom_level=1.2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))